# Creación de un modelo de sentimientos.

El modelo de sentimientos, ha sido necesario crear un dataset hecho a mano analizando las frases con la libreria nltk y despues se entrena el modelo a traves del dataset previamente creado.

## Importaciones y instalaciones.

In [1]:
!pip install translate

In [2]:
pip install tensorflow

In [3]:
pip install keras

In [25]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.chat.util import Chat, reflections
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from translate import Translator
import requests
from bs4 import BeautifulSoup
import re
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torchsummary import summary
import tensorflow as ts
from tensorflow import keras
import itertools
from sklearn.metrics import precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
from tensorflow.keras.callbacks import History, EarlyStopping

# Descarga los recursos de nltk necesarios
nltk.download('vader_lexicon')
nltk.download('stopwords')



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Creo un buscador de sinonimos.

Aquí estoy buscando sinonimos de palabras para despues penalizarlas o puntuarlas mejor.

In [5]:
sinonimos_conjunto = []
sinonimos_lista = []
def obtener_sinonimos(palabra):
    global sinonimos_conjunto

    url = 'http://www.wordreference.com/sinonimos/'
    enlace = f"{url}{palabra}"

    resp = requests.get(enlace)
    bs = BeautifulSoup(resp.text, 'html.parser')

    # Busca la sección de sinónimos y antónimos
    seccion_sinonimos_antonimos = bs.find('div', id='otherDicts').find('div', class_='trans esp clickable')

    # Verifica si se encontró la sección de sinónimos y antónimos
    if seccion_sinonimos_antonimos:
        # Extrae y agrega los sinónimos a la lista
        for elemento in seccion_sinonimos_antonimos.find_all(['li', 'div']):
            texto = elemento.get_text(strip=True)
            if texto.startswith('Antónimos:'):
                break  # Termina si encontramos la sección de antónimos
            sinonimos_conjunto.extend(texto.split(', '))

        # Imprime los sinónimos para la palabra actual
        sinonimos_conjunto += sinonimos_lista
        print(f"Sinónimos de '{palabra}': {sinonimos_conjunto}")

    else:
        print(f"No se encontraron sinónimos para la palabra '{palabra}'.")

# Lista de palabras para buscar sinónimos
palabras_a_buscar = ["triste", "morir","perdido","ansiedad"]

# Itera sobre la lista y obtén sinónimos para cada palabra
for palabra in palabras_a_buscar:
    obtener_sinonimos(palabra)

# Convierte la lista a un string antes de imprimir
print("Conjuntas: " + str(sinonimos_conjunto))

Sinónimos de 'triste': ['afligido', ' apenado', ' entristecido', ' apesadumbrado', ' atribulado', ' pesaroso', ' mohíno', ' mustio', ' taciturno', ' compungido', ' lloroso', ' cariacontecido']
Sinónimos de 'morir': ['afligido', ' apenado', ' entristecido', ' apesadumbrado', ' atribulado', ' pesaroso', ' mohíno', ' mustio', ' taciturno', ' compungido', ' lloroso', ' cariacontecido', 'fallecer', ' expirar', ' agonizar', ' sucumbir', ' perecer', ' fenecer', ' finar', ' diñar', ' espichar', 'estirar la pata', ' pasar a mejor vida', ' irse al otro barrio']
Sinónimos de 'perdido': ['afligido', ' apenado', ' entristecido', ' apesadumbrado', ' atribulado', ' pesaroso', ' mohíno', ' mustio', ' taciturno', ' compungido', ' lloroso', ' cariacontecido', 'fallecer', ' expirar', ' agonizar', ' sucumbir', ' perecer', ' fenecer', ' finar', ' diñar', ' espichar', 'estirar la pata', ' pasar a mejor vida', ' irse al otro barrio', 'olvidado', ' extraviado', ' abandonado', ' descuidado', ' despistado', ' d

## Función que para puntualizar las frases.

En este codigo estoy haciendo una puntuacion a una frase comprobando si tiene palabras negativas para puntualizarla peor o si tiene palabras positivas para puntualizarlas mas.

In [6]:
def get_sentiment_score(text):
    translator = Translator(to_lang='en', from_lang='es')
    translation = translator.translate(text)
    text_en = translation

    sia = SentimentIntensityAnalyzer()

    # Obtener la polaridad del análisis de sentimientos
    sentiment_polarity = sia.polarity_scores(text_en)['compound']

    # Ajustar manualmente el valor para considerar palabras positivas
    positive_words = ["si", "sí", "afirmativo", "claro"]  # Puedes ampliar esta lista según tus necesidades
    negative_words = sinonimos_conjunto  # Asegúrate de definir este conjunto

    # Factor de ajuste adicional para palabras positivas
    positive_factor = 0.0

    # Factor de ajuste adicional para palabras negativas
    negative_factor = 0.0

    # Sumar positive_factor solo si alguna palabra positiva está presente
    if any(word in text.lower() for word in positive_words):
        positive_factor += abs(sentiment_polarity) * 0.1  # Puedes ajustar este valor según tus necesidades

    # Sumar negative_factor solo si alguna palabra negativa está presente
    if any(word in text.lower() for word in negative_words):
        negative_factor += abs(sentiment_polarity) * 0.1  # Puedes ajustar este valor según tus necesidades

    # Calcular el puntaje final del sentimiento
    sentiment_score = max(0, (sentiment_polarity + 1) * 2.5 + positive_factor - negative_factor)

    return text_en, sentiment_score

## Comprobacion individual

In [7]:
texto = "esto harto de proyectos quiero acabar con mi vida "

# Dividir el texto en palabras
palabras = texto.split()

lista_de_palabras=["super","bastante"]
contiene_palabra_delante=False

indice_cantidad=0
indice_negativo=0
# Iterar sobre las palabras
for indice,palabra in enumerate(palabras):
    for palabra1 in lista_de_palabras:
      if palabra1==palabra:
        indice_cantidad=indice

    resultado,score = get_sentiment_score(palabra)
    if score < 2.5:
      indice_negativo=indice

    if indice_cantidad == (indice_negativo-1):
      contiene_palabra_delante=True

respuesta,score = get_sentiment_score(texto)
if contiene_palabra_delante == True:
  score=score/2
print(f"El sentimiento de es: ",score)

El sentimiento de es:  0.77275


Como podemos ver esta dandole una puntuación adecuada a la frase.

## Funcion para pasarle un array de frases y que las puntualice.

In [8]:
def funcion(texto):
  # Dividir el texto en palabras
  palabras = texto.split()

  lista_de_palabras=["super","bastante"]
  contiene_palabra_delante=False

  indice_cantidad=0
  indice_negativo=0
  # Iterar sobre las palabras
  for indice,palabra in enumerate(palabras):
      for palabra1 in lista_de_palabras:
        if palabra1==palabra:
          indice_cantidad=indice

      resultado,score = get_sentiment_score(palabra)
      if score < 2.5:
        indice_negativo=indice

      if indice_cantidad == (indice_negativo-1):
        contiene_palabra_delante=True

  respuesta,score = get_sentiment_score(texto)
  if contiene_palabra_delante == True:
    score=score/2

  return score

## Arreglando las frases antes de puntualizarlas.

In [9]:
def limpiar_texto(texto):
    # Convertir el texto a minúsculas
    texto_minusculas = texto.lower()

    # Eliminar los patrones de arroba seguido de texto hasta el espacio
    texto_sin_arrobas = re.sub(r'@\S+', '', texto_minusculas)

    # Eliminar las URLs
    texto_sin_urls = re.sub(r'http[s]?://\S+', '', texto_sin_arrobas)

    # Eliminar los emoticonos
    texto_sin_emoticonos = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+', '', texto_sin_urls)

    # Eliminar los hashtags y lo que sigue hasta el espacio
    texto_sin_hashtags = re.sub(r'#\S+\s', '', texto_sin_emoticonos)

    return texto_sin_hashtags


# Las frases proporcionadas
textos = [
    "Un chico bohemio para fumar, leer, follar y hablar",
    "Ya me acostumbré a despertarme y no tener ni un mensaje en WhatsApp",
    "Si no apoyamos a los nuestros no estamos en nada, en pocos minutos #ElRingdelaspalabras en #Larfm https://t.co/2ucJSiVFit",
    "Engomado con la lectura erótica.",
    "Una gran pérdida para la música, para el arte. https://t.co/atCh1xyS05",
    "@Ritsuka4 Igual tú para mí 🤗",
    "Borracho.",
    "@JeffCaicedo 😍",
    "¿Eres feliz?",
    "Yo escribo aquí como para dejar algunas cosas que pienso en el momento porque me leen como 3 personas, los amo igual. 😍",
    "Con el tiempo entendí que a los que llamé amigos o mejores amigos solo lo fueron de una época de mi vida, están en un lugar especial pero ya no somos las mismas personas que éramos antes.",
    "Ay querido si siempre vas a abrir la boca para destilar veneno ahógate tú solo en tus palabras.... Yo me voy, ahí te quedas jaja",
    "Me gusta estar al lado del camino fumando el humo mientras todo pasa",
    "Muero por un beso de esos que no son de amigos",
    "@MariaCamila_C Jajaja bebé eso termina en el suelo, las arruncho mientras me duermo pero ya luego me fastidian",
    "Soy de amores simples, de tragos corrientes en la calle sin apariencias y abriendo los sentimientos hablando de lo que realmente vale sin aparentar cuánto pagamos por lo que nos tomamos.",
    "Vengo a confesarles que me hace mucha falta la mariconería, bailar así muy entregado en el antro con las amigas maricas 🏳️‍🌈🏳️‍🌈🏳️‍🌈🏳️‍🌈",
    "@MariaCamila_C Tengo 4 pero dejo solo unita",
    "Si usa gel déjalo ir 😂",
    "Querer superar la depre pero no dejar de poner temas sad. Así es",
    "Sueño con el día que pueda ver sin necesidad de ponerme las gafas.",
    "Ignorar ignorar ignorar.",
    "Amanecí fastidiado de lo que esperan siempre que sea y haga.",
    "Tengo un toque Midas para las relaciones, cualquiera que medio se acerca a un intento de algo conmigo fijo fijo consigue su relación soñada después de mí.",
    "Es difícil no sentirse frustrado viviendo en un país en el que las oportunidades son tan escasas, aún así de verdad espero esta vez conseguir algo de trabajo. Ojalá ustedes tengan uno bueno 💪"
]

textos_procesados = [limpiar_texto(texto) for texto in textos]
array=[]
textos_procesados = []

for texto in textos:
    texto_procesado = limpiar_texto(texto)
    textos_procesados.append(texto_procesado)

print("textos =", textos_procesados)


textos = ['un chico bohemio para fumar, leer, follar y hablar', 'ya me acostumbré a despertarme y no tener ni un mensaje en whatsapp', 'si no apoyamos a los nuestros no estamos en nada, en pocos minutos en ', 'engomado con la lectura erótica.', 'una gran pérdida para la música, para el arte. ', ' igual tú para mí ', 'borracho.', ' ', '¿eres feliz?', 'yo escribo aquí como para dejar algunas cosas que pienso en el momento porque me leen como 3 personas, los amo igual. ', 'con el tiempo entendí que a los que llamé amigos o mejores amigos solo lo fueron de una época de mi vida, están en un lugar especial pero ya no somos las mismas personas que éramos antes.', 'ay querido si siempre vas a abrir la boca para destilar veneno ahógate tú solo en tus palabras.... yo me voy, ahí te quedas jaja', 'me gusta estar al lado del camino fumando el humo mientras todo pasa', 'muero por un beso de esos que no son de amigos', ' jajaja bebé eso termina en el suelo, las arruncho mientras me duermo pero ya lu

Aqui estamos eliminadon valores y caracteres que no son necesarios, como los iconos, eliminar los @ y el texto asociado, eliminar # y su texto, eliminar urls y convertir todo a minusculas.

## Puntualizando frases para añadirlo al dataset.

In [11]:
textos = ['esta soy yo al ver que las directioners piden ayuda a otros fandoms y la pagina no cuenta todo lo que ya llevamos que creo que arrevazamos a 1d pero la pagina ni verga  ', 'púdrete maldita sensación de angustia y desespero por no tener nada estable, déjame descansar de tu compañía pinche perra.']

for resultado in textos:
  score=funcion(resultado)
  formatted_score = "{:.2f}".format(score)
  print(resultado," ,puntuacion: ", formatted_score)


esta soy yo al ver que las directioners piden ayuda a otros fandoms y la pagina no cuenta todo lo que ya llevamos que creo que arrevazamos a 1d pero la pagina ni verga    ,puntuacion:  3.04
púdrete maldita sensación de angustia y desespero por no tener nada estable, déjame descansar de tu compañía pinche perra.  ,puntuacion:  0.02


Podemos ver que esta puntualizando correctamente las frases, estas frases se añadiria a un excel borrando las palabras individuales y borrando frases en blanco.
Una vez que tengamos suficientes frases crearemos el csv.

## Importamos el csv que hemos creado.

In [13]:
file_path ='/content/drive/MyDrive/dataset/dataset_sentimientos.csv'
df = pd.read_csv(file_path)

## Muestras de informacion del dataset.

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   texto       381 non-null    object 
 1   puntuacion  381 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.1+ KB


Podemos ver que tenemos 343 registros, que no tenemos valores nulos y tenemos dos columnas:
- Texto: frase que se a puntualizado
- Puntuacion: puntuación de la frase

In [15]:
df.head()

,texto,puntuacion
0,No me afecta,2.50
1,Angustia,1.00
2,No me importa,3.73
3,No me afecta porque no voy por detrás,2.50
4,"Mal, me pongo triste y no me gusta porque son ...",0.51


Podemos ver los 5 primeros resultados

In [16]:
df.describe()

,puntuacion
count,381.000000
mean,2.690315
std,1.053938
min,0.010000
25%,2.120000
50%,2.900000
75%,3.350000
max,4.840000


Vemos que el valor minimo es un 0.01, el maximo 4,84 y la media 2,69. Tenemos unos buenos datos.

## Preparacion de los modelos y entrenamiento.

In [17]:
# Con las estopwords
df['texto'] = df['texto'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word.isalpha()]))

In [18]:
# Sin las stopwords
stop_words = set(stopwords.words('spanish'))
df['texto'] = df['texto'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word.isalpha() and word.lower() not in stop_words]))

Probamos como funcionan mejor los modelos si dejando las stopword o quitandolas.

### Separamos los valores para el entrenamiento.

In [19]:
# División de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['texto'], df['puntuacion'], test_size=0.2, random_state=42)

### Modelo SVR.

In [20]:
# Crear un modelo de Support Vector Machine con un pipeline que incluye un vectorizador de texto
model_svm = make_pipeline(CountVectorizer(), SVR())

# Entrenar el modelo SVM
model_svm.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predictions_svm = model_svm.predict(X_test)

# Evaluar el rendimiento del modelo SVM (opcional)
mae_svm = mean_absolute_error(y_test, predictions_svm)
print(f'Mean Absolute Error (SVM): {mae_svm}')

# Calcular el rango de los datos
rango_datos = np.ptp(y_train)
print(f'Rango de los datos: {rango_datos}')

# Calcular el porcentaje de precisión
precision = 1 - mae_svm / rango_datos
print(f'Porcentaje de precisión: {precision * 100:.2f}%')

Mean Absolute Error (SVM): 0.8328487949807007
Rango de los datos: 4.83
Porcentaje de precisión: 82.76%


### Modelo de regresión lineal.

In [21]:
# Crear un modelo de regresión lineal con un pipeline que incluye un vectorizador de texto
model = make_pipeline(CountVectorizer(), LinearRegression())

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predictions = model.predict(X_test)

# Evaluar el rendimiento del modelo (opcional)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Calcular el rango de los datos
rango_datos = np.ptp(y_train)

# Calcular el Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)

# Calcular el porcentaje de precisión
precision = 1 - mse / (rango_datos ** 2)

print(f'Porcentaje de precisión: {precision * 100:.2f}%')

Mean Squared Error: 1.7861577832948043
Porcentaje de precisión: 92.34%


### Modelo RandomForestRegressor.

In [22]:
# Crear un modelo de Random Forest con un pipeline que incluye un vectorizador de texto
model_rf = make_pipeline(CountVectorizer(), RandomForestRegressor())

# Entrenar el modelo Random Forest
model_rf.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predictions_rf = model_rf.predict(X_test)

# Evaluar el rendimiento del modelo Random Forest (opcional)
mae_rf = mean_absolute_error(y_test, predictions_rf)
print(f'Mean Absolute Error (Random Forest): {mae_rf}')

# Calcular el rango de los datos
rango_datos = np.ptp(y_train)
print(f'Rango de los datos: {rango_datos}')

# Calcular el porcentaje de precisión
precision = 1 - mae_rf / rango_datos
print(f'Porcentaje de precisión: {precision * 100:.2f}%')

Mean Absolute Error (Random Forest): 0.7867273084210821
Rango de los datos: 4.83
Porcentaje de precisión: 83.71%


### Modelo red neuronal.

In [26]:
# Tokenización y padding para la entrada de la red neuronal
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1
max_length = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_test_seq))
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Crear el modelo de red neuronal
model_nn = Sequential()
model_nn.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length))
model_nn.add(Flatten())
model_nn.add(Dense(500, activation='relu'))
model_nn.add(Dropout(0.05))
model_nn.add(Dense(500, activation='relu'))
model_nn.add(Dropout(0.05))
model_nn.add(Dense(500, activation='relu'))
model_nn.add(Dropout(0.05))
model_nn.add(Dense(500, activation='relu'))
model_nn.add(Dropout(0.05))
model_nn.add(Dense(500, activation='relu'))
model_nn.add(Dropout(0.05))
model_nn.add(Dense(1, activation='linear'))

# Compilar el modelo con learning rate personalizado
model_nn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')  # Para problemas de regresión

# Definir el callback History para almacenar métricas
history_callback = History()

# Entrenar el modelo con EarlyStopping y History
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_nn.fit(X_train_pad, y_train, epochs=25, batch_size=32, validation_data=(X_test_pad, y_test), callbacks=[early_stopping, history_callback])

# Obtener las métricas de entrenamiento y validación
training_metrics = history_callback.history['loss']
validation_metrics = history_callback.history['val_loss']

# Realizar predicciones en el conjunto de prueba
predictions_nn = model_nn.predict(X_test_pad)

# Evaluar el rendimiento del modelo de red neuronal (opcional)
mae_nn = mean_absolute_error(y_test, predictions_nn)
print(f'Mean Absolute Error (Neural Network): {mae_nn}')

# Calcular el rango de los datos
rango_datos = np.ptp(y_train)
print(f'Rango de los datos: {rango_datos}')

# Calcular el porcentaje de precisión
precision = 1 - mae_nn / rango_datos
print(f'Porcentaje de precisión: {precision * 100:.2f}%')

Epoch 1/25
10/10 [==============================] - 2s 51ms/step - loss: 7.0906 - val_loss: 4.5270
Epoch 2/25
10/10 [==============================] - 0s 26ms/step - loss: 2.3337 - val_loss: 2.2872
Epoch 3/25
10/10 [==============================] - 0s 26ms/step - loss: 1.4334 - val_loss: 1.4342
Epoch 4/25
10/10 [==============================] - 0s 26ms/step - loss: 1.3200 - val_loss: 1.3456
Epoch 5/25
10/10 [==============================] - 0s 26ms/step - loss: 1.1578 - val_loss: 1.3682
Epoch 6/25
10/10 [==============================] - 0s 27ms/step - loss: 1.1221 - val_loss: 1.3357
Epoch 7/25
10/10 [==============================] - 0s 26ms/step - loss: 1.0790 - val_loss: 1.3353
Epoch 8/25
10/10 [==============================] - 0s 27ms/step - loss: 1.0530 - val_loss: 1.3331
Epoch 9/25
10/10 [==============================] - 0s 26ms/step - loss: 1.0519 - val_loss: 1.3324
Epoch 10/25
10/10 [==============================] - 0s 29ms/step - loss: 1.0354 - val_loss: 1.3310
Epoch 11/

## Resultado de los modelos y pruebas

In [27]:
new_texts = ['estoy super contento','estoy bastante mal','me quiero morir','se me a juntado la tristeza y la depresion']
stop_words = set(stopwords.words('spanish'))
#new_texts_processed = [' '.join([word.lower() for word in x.split() if word.isalpha() and word.lower() not in stop_words]) for x in new_texts]

new_predictions = model.predict(new_texts)
print("Modelo 1: ",new_predictions)

new_predictions = model_svm.predict(new_texts)
print("Modelo 2: ",new_predictions)

new_predictions_rf = model_rf.predict(new_texts)
print("Modelo 3: ",new_predictions_rf)

new_texts_seq = tokenizer.texts_to_sequences(new_texts)
new_texts_pad = pad_sequences(new_texts_seq, maxlen=max_length, padding='post')
new_predictions_nn = model_nn.predict(new_texts_pad)
print("Modelo 4: ",new_predictions_nn)

Modelo 1:  [4.23474756 1.22759388 2.5672174  2.51917175]
Modelo 2:  [3.17500356 1.83105475 2.61806822 2.52757128]
Modelo 3:  [3.63373333 1.39414103 2.55350997 2.53998038]
1/1 [==============================] - 0s 26ms/step
Modelo 4:  [[2.701569 ]
 [2.6394732]
 [2.6639829]
 [2.643667 ]]


Aquí podemos ver algunos resultados con algunas frases el que mayor coerencia tiene pensamos que es el de LinearRegression.

## Exportamos el modelo.

In [ ]:
joblib.dump(model,"/content/drive/MyDrive/dataset/sentimientos_modelo.pkl")